## Assignment 2: MongoDB

This assignment is based on content discussed in the Introduction to MongoDB module.

## Learning outcomes 

The purpose of this assignment is for learners to be able to:
- Familarize with JSON document syntax
- Understand basic MongoDB CRUD operations
- Understand MongoDB data pipelines to run aggregate queries

In this assignment, you will make use of the sample data provided.  

This dataset has 3 collections: Employee, Workplace and Address.  You will import this data into your local MongoDB database.

Required imports for this project are given below. Make sure you have all libraries required for this project installed. You may use conda or pip based on your set up.

## Setup Notes

**Please note** These instructions are duplicated here for your reference. You would have encountered them in a previous module. It is expected that you have already taken the steps to set up and run MongoDB.

We will be using MongoDB Community Edition.  The MongoDB database <b>MUST</b> be installed and running locally before continuing with this notebook.  We will need to install two packages using the Anaconda package manager:

1. [Mongodb](https://www.mongodb.com/) - this package contains the mongodb database 
2. [PyMongo](https://www.mongodb.com/docs/drivers/pymongo/) - this package contains the python driver that will allow us to communicate with the mongodb database.

#### Install

1. Open a command line terminal and execute the following command to install mongodb.
```console
conda install -c anaconda mongodb
```
2. Open a command line terminal and execute the following command to install pymongo packge.
```console
conda install -c anaconda pymongo
```

#### Run Mongodb in Windows
1. MongoDB requires a data directory to store all data. MongoDB’s default data directory path is \data\db. Create this folder using the following commands from a Command Prompt:
```console
md \data\db
```

2. To start MongoDB, run mongod.exe. For example, from the Command Prompt:
```console
"C:\Program Files\MongoDB\Server\3.2\bin\mongod.exe"
```

#### Run Mongodb in Mac
1. MongoDB requires a data directory to store all data. MongoDB’s default data directory path is /data/db. Create this folder using the following commands from a Command Prompt.  Note that we run the command as a super user using the "sudo" command:
```console
sudo mkdir /data/db/
```

2. To start MongoDB, run mongod.exe. For example, from the Command Promp.  Note that we run the command as a super user using the "sudo" command:
```console
sudo mongod --config /usr/local/etc/mongod.conf
```

In [1]:
#required imports
import os
import json
import datetime
import pymongo
import pprint
import pandas as pd
import numpy as np
from pymongo import MongoClient
from bson import ObjectId
print('Mongo version', pymongo.__version__)

Mongo version 3.12.0


We first need to connect to our locally running MongoDB database (<b>Make sure your database is running on your machine</b>). We will use the MongoClient to connect to a local 'test' database that is running on port 27017 (this is the default port).

In [2]:
client = MongoClient('localhost', 27017)
db = client.assignment1

After installing necessary modules proceed to import the data into your database.

In [3]:
# Let's delete any existing collections in our database
db.workplace.drop()
db.address.drop()
db.employee.drop()

# Import our files into our three collections
with open('Data/Employee.json') as f:
    db.employee.insert_many(json.load(f))
with open('Data/Workplace.json') as f:
    db.workplace.insert_many(json.load(f))
with open('Data/Address.json') as f:
    db.address.insert_many(json.load(f))

#### Question 1 (10 Marks)

The address collection contains employee from different ages and interests.  Perform a simple query to list all employees that are less than or equal to 50 and like Cooking.

__NOTE:__ the following shows the structure of an Employee document that will help you construct the query.

In [4]:
pprint.pprint(client.assignment1.employee.find_one())

{'_id': '9f39da36-82cc-4353-ab90-d616105fa7c1',
 'address_id': 'b6c0b50a-d0e3-43bf-a2a4-8d4674c2a7e8',
 'age': 40,
 'email': 'ih@ri.ro',
 'firstname': 'Emilie',
 'interests': ['Bowling', 'Cooking', 'Golf', 'Swimming'],
 'lastname': 'Woods',
 'workplace_id': 'a32bf18d-e0e5-48f2-a851-aa49c80f9460'}


In [5]:
# YOUR CODE HERE
for employee in client.assignment1.employee.find({'age':{'$lte':50},'interests':'Cooking'}):
    pprint.pprint(employee)

{'_id': '9f39da36-82cc-4353-ab90-d616105fa7c1',
 'address_id': 'b6c0b50a-d0e3-43bf-a2a4-8d4674c2a7e8',
 'age': 40,
 'email': 'ih@ri.ro',
 'firstname': 'Emilie',
 'interests': ['Bowling', 'Cooking', 'Golf', 'Swimming'],
 'lastname': 'Woods',
 'workplace_id': 'a32bf18d-e0e5-48f2-a851-aa49c80f9460'}
{'_id': 'af27265e-6639-49f2-991e-193275a4111a',
 'address_id': '64fd714d-e219-4e45-888b-cc2238a8bd0b',
 'age': 18,
 'email': 'sug@gon.bf',
 'firstname': 'Thomas',
 'interests': ['Cooking', 'Cricket', 'Tennis', 'Swimming', 'Fishing'],
 'lastname': 'Patterson',
 'workplace_id': '5345fcb9-6297-4b9f-aa15-cbee8460f28f'}
{'_id': '00289d48-bad8-4b73-a359-a1a1f05c96e2',
 'address_id': '8a430805-00b8-40a6-bd93-c950b544a83b',
 'age': 22,
 'email': 'ra@dupnejuk.nr',
 'firstname': 'Sophia',
 'interests': ['Hiking', 'Soccer', 'Bowling', 'Rubgy', 'Cooking', 'Dancing'],
 'lastname': 'Flores',
 'workplace_id': 'b12cd444-e65b-4bc2-8cf6-2dbe854a627b'}
{'_id': 'da76e52b-b3db-4fc0-b0d6-435d1aed0cd9',
 'address_id

#### Question 2  (10 Marks)

Insert a new Employee with the following properties:

* First Name: Jake 
* Last Name: Sample
* Email: jakesample@email.com
* Age: 26
* Interest: Biking, Hiking

Also, this employee works for 'Union Planters Corp' and lives at '573 Wojhas Square, Victoria'.
Verify that the insert succeeded and display the generated employees _id attribute.

__HINT__ An Employee document references a Workplace and Address document

In [6]:
# Check number of documents in collection prior to insert
client.assignment1.employee.count_documents({})

100

In [7]:
# YOUR CODE HERE
# Insert new Employee
emp_address_id=client.assignment1.address.find_one({'address':'573 Wojhas Square'})['_id']
emp_workplace_id=client.assignment1.workplace.find_one({'name':'Union Planters Corp'})['_id']

client.assignment1.employee.insert_one({'firstname':'Jake','lastname':'Sample','email':'jakesample@email.com','age':26,'interests':['Biking','Hiking'],'address_id':emp_address_id,'workplace_id':emp_workplace_id})

In [8]:
# Check number of documents in collection after insert
client.assignment1.employee.count_documents({})

101

In [9]:
# Verify the insert
for employee in client.assignment1.employee.find({'firstname':'Jake','lastname':'Sample'}):
    pprint.pprint(employee)

{'_id': ObjectId('652caee7b8960f343676c69f'),
 'address_id': '91b5b7b3-2309-4e8a-8247-cd66d626ef0c',
 'age': 26,
 'email': 'jakesample@email.com',
 'firstname': 'Jake',
 'interests': ['Biking', 'Hiking'],
 'lastname': 'Sample',
 'workplace_id': '5345fcb9-6297-4b9f-aa15-cbee8460f28f'}


#### Question 3 (10 Marks)

Delete all employees that work for 'Great Plains Energy Inc.' and are greater than 46 years old and likes 'Tennis'. Once you delete the employees verify the number of employees deleted.

In [10]:
# YOUR CODE HERE
# Check number of documents in collection prior to delete
client.assignment1.employee.count_documents({})

101

In [11]:
# First find the documents that meet the criteria listed
emp_workplace_id2=client.assignment1.workplace.find_one({'name':'Great Plains Energy Inc.'})['_id']
for employee in client.assignment1.employee.find({'age':{'$gt':46},'interests':'Tennis','workplace_id':emp_workplace_id2}):
    pprint.pprint(employee)

{'_id': '153ba346-b33d-4611-802c-fd95c7cb078c',
 'address_id': '8264af1e-1c55-4ed8-96f9-e3b9e1eff7d5',
 'age': 71,
 'email': 'ov@sutec.mr',
 'firstname': 'Adelaide',
 'interests': ['Tennis'],
 'lastname': 'Curry',
 'workplace_id': 'a32bf18d-e0e5-48f2-a851-aa49c80f9460'}
{'_id': '8e082f05-13b7-485d-89c4-259f1103f95c',
 'address_id': '62996de2-fcda-4894-95d5-fef0d11d92f4',
 'age': 48,
 'email': 'wi@uvafej.pt',
 'firstname': 'Lester',
 'interests': ['Bowling', 'Tennis'],
 'lastname': 'Summers',
 'workplace_id': 'a32bf18d-e0e5-48f2-a851-aa49c80f9460'}
{'_id': 'dfce1d98-15f6-4b75-86f6-cd5d05cfdc04',
 'address_id': '2987cea4-b71d-4338-b0ef-a9b5feb3a138',
 'age': 80,
 'email': 'vukhik@wejoce.nf',
 'firstname': 'Sadie',
 'interests': ['Golf', 'Fishing', 'Bowling', 'Dancing', 'Cooking', 'Tennis'],
 'lastname': 'Park',
 'workplace_id': 'a32bf18d-e0e5-48f2-a851-aa49c80f9460'}
{'_id': '86417f71-18cd-41c5-ad01-24b31ff2074f',
 'address_id': 'fe80c95a-a0e9-42c9-aa6a-56f27020842d',
 'age': 65,
 'email

In [12]:
# Delete employees that meet the critieria listed
client.assignment1.employee.delete_many({'age':{'$gt':46},'interests':'Tennis','workplace_id':emp_workplace_id2})

In [13]:
# Verify that these documents have been deleted
for employee in client.assignment1.employee.find({'age':{'$gt':46},'interests':'Tennis','workplace_id':emp_workplace_id2}):
    pprint.pprint(employee)

In [14]:
# Verify new total number of documents in the collection
# Should be 101-4=97
client.assignment1.employee.count_documents({})

97

#### Question 4 (12 Marks)
Add a new field called 'industry' to all employees that work for 'Health Net Inc.' and populate the field with the value 'Health Care'.

__HINT__ Adding a new field to a document is like updating the document

In [15]:
# YOUR CODE HERE
# First find the documents that meet the criteria listed
emp_workplace_id3=client.assignment1.workplace.find_one({'name':'Health Net Inc.'})['_id']
for employee in client.assignment1.employee.find({'workplace_id':emp_workplace_id3}):
    pprint.pprint(employee)
client.assignment1.employee.count_documents({'workplace_id':emp_workplace_id3})

{'_id': 'ad4fd8bd-b732-4388-8469-5395b497df63',
 'address_id': '91b32548-65b6-4fd9-91a1-2d86b04b8eea',
 'age': 42,
 'email': 'tasnimnag@tefifdum.bo',
 'firstname': 'Marc',
 'interests': ['Golf', 'Cricket', 'Rubgy', 'Hiking', 'Fishing', 'Dancing'],
 'lastname': 'Castro',
 'workplace_id': 'a222385c-342c-43ea-adbc-9b487a2ee2be'}
{'_id': '6222b4f5-6fd3-4c72-b058-710fb568b869',
 'address_id': '59516270-19f4-47e8-95ae-3e83fa5f230c',
 'age': 46,
 'email': 'ruupeke@oh.om',
 'firstname': 'Lucinda',
 'interests': ['Golf', 'Rubgy', 'Tennis'],
 'lastname': 'Gomez',
 'workplace_id': 'a222385c-342c-43ea-adbc-9b487a2ee2be'}
{'_id': 'ca189ae9-d04b-4ae0-9b4a-b725196c31d2',
 'address_id': '16ff1778-b687-42ef-aace-36cedc05f521',
 'age': 37,
 'email': 'vom@calfim.se',
 'firstname': 'Emilie',
 'interests': ['Hiking'],
 'lastname': 'Robertson',
 'workplace_id': 'a222385c-342c-43ea-adbc-9b487a2ee2be'}
{'_id': 'ed499274-f17f-4a4c-8327-7e740e6dd9c1',
 'address_id': '01185f9b-b00f-4f2d-9878-0c989fe00fbc',
 'age

14

In [16]:
# Perform update
client.assignment1.employee.update_many({'workplace_id':emp_workplace_id3},{'$set':{'industry':'Health Care'}})

In [17]:
# Verify update
for employee in client.assignment1.employee.find({'workplace_id':emp_workplace_id3}):
    pprint.pprint(employee)
client.assignment1.employee.count_documents({'workplace_id':emp_workplace_id3})

{'_id': 'ad4fd8bd-b732-4388-8469-5395b497df63',
 'address_id': '91b32548-65b6-4fd9-91a1-2d86b04b8eea',
 'age': 42,
 'email': 'tasnimnag@tefifdum.bo',
 'firstname': 'Marc',
 'industry': 'Health Care',
 'interests': ['Golf', 'Cricket', 'Rubgy', 'Hiking', 'Fishing', 'Dancing'],
 'lastname': 'Castro',
 'workplace_id': 'a222385c-342c-43ea-adbc-9b487a2ee2be'}
{'_id': '6222b4f5-6fd3-4c72-b058-710fb568b869',
 'address_id': '59516270-19f4-47e8-95ae-3e83fa5f230c',
 'age': 46,
 'email': 'ruupeke@oh.om',
 'firstname': 'Lucinda',
 'industry': 'Health Care',
 'interests': ['Golf', 'Rubgy', 'Tennis'],
 'lastname': 'Gomez',
 'workplace_id': 'a222385c-342c-43ea-adbc-9b487a2ee2be'}
{'_id': 'ca189ae9-d04b-4ae0-9b4a-b725196c31d2',
 'address_id': '16ff1778-b687-42ef-aace-36cedc05f521',
 'age': 37,
 'email': 'vom@calfim.se',
 'firstname': 'Emilie',
 'industry': 'Health Care',
 'interests': ['Hiking'],
 'lastname': 'Robertson',
 'workplace_id': 'a222385c-342c-43ea-adbc-9b487a2ee2be'}
{'_id': 'ed499274-f17f-4

14

In [18]:
# Document count did not change after update which is what we expected
client.assignment1.employee.count_documents({})

97

#### Question 5 (10 Marks)

Create an aggregate query to count the number of employees for each company and sort the output from largest employee count to lowest employee count.

__NOTE__ you will use a pipeline to achieve the computed result.  You should produce a result similar to the following table (the following table contains fake data)
<table>
    <tr><th></th><th>_id</th><th>count</th></tr>
    <tr><td>0</td><td>[Equity Residential Properties Trust]</td><td>19</td></tr>
    <tr><td>...</td><td>...</td><td>...</td></tr>
    <tr><td>7</td><td>[Bell Microproducts Inc.]</td><td>6</td></tr>
    <tr><td>8</td><td>[Kemet Corp.]</td><td>1</td></tr>
</table>

__HINT__ you should make use of the \\$lookup, \\$group and \\$sort pipeline operations

In [19]:
# First view the names of the workplaces in the collection
for workplace in client.assignment1.workplace.find({},{'_id':0,'name':1}):
    pprint.pprint(workplace)

{'name': 'Union Planters Corp'}
{'name': 'Aetna Inc.'}
{'name': 'Health Net Inc.'}
{'name': 'Kemet Corp.'}
{'name': 'Bell Microproducts Inc.'}
{'name': 'Equity Residential Properties Trust'}
{'name': 'Equity Office Properties Trust'}
{'name': 'Hilton Solutions'}
{'name': 'Great Plains Energy Inc.'}
{'name': 'Xcel Bear Inc'}
{'name': 'Novellus Systems Inc'}
{'name': 'Citizens Hawk'}


In [20]:
# Check output of names of workplaces and count of employees at each
for employee in client.assignment1.employee.aggregate([
    {
    '$lookup':
       {
         'from':'workplace',
         'localField':'workplace_id',
         'foreignField':'_id',
         'as':'workplace_name'
        }   
    },
    {
        '$unwind':'$workplace_name'
    },
    {
    '$group':  
        {'_id': '$workplace_name.name',  
         'count':{'$sum':1} 
        }
    },
    {
    '$sort':{'count':-1} 
    }
    ]):
 
    pprint.pprint(employee)


{'_id': 'Hilton Solutions', 'count': 15}
{'_id': 'Health Net Inc.', 'count': 14}
{'_id': 'Aetna Inc.', 'count': 13}
{'_id': 'Bell Microproducts Inc.', 'count': 11}
{'_id': 'Equity Office Properties Trust', 'count': 10}
{'_id': 'Union Planters Corp', 'count': 10}
{'_id': 'Equity Residential Properties Trust', 'count': 7}
{'_id': 'Xcel Bear Inc', 'count': 6}
{'_id': 'Kemet Corp.', 'count': 6}
{'_id': 'Great Plains Energy Inc.', 'count': 5}


In [21]:
# YOUR CODE HERE
# Run aggregation pipeline and produce dataframe

query_result = client.assignment1.employee.aggregate([
    {
    '$lookup':
       {
         'from':'workplace',
         'localField':'workplace_id',
         'foreignField':'_id',
         'as':'workplace_name'
        }   
    },
    {
        '$unwind':'$workplace_name'
    },
    {
    '$group':  
        {'_id': '$workplace_name.name',  
         'count':{'$sum':1} 
        }
    },
    {
    '$sort':{'count':-1} 
    }
    ])

df=pd.DataFrame(list(query_result))
df

,_id,count
0,Hilton Solutions,15
1,Health Net Inc.,14
2,Aetna Inc.,13
3,Bell Microproducts Inc.,11
4,Equity Office Properties Trust,10
5,Union Planters Corp,10
6,Equity Residential Properties Trust,7
7,Xcel Bear Inc,6
8,Kemet Corp.,6
9,Great Plains Energy Inc.,5
